<a href="https://colab.research.google.com/github/MichalAdamski1/uczenie-maszynowe-w-finansach/blob/main/case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter

df = pd.read_csv('creditcard.csv')

print("Liczba brakujących wartości:", df.isna().sum().sum())

print("Rozkład klas:\n", df['Class'].value_counts())

X = df.drop(columns=['Class'])
y = df['Class']

scaler = StandardScaler()
X['Amount'] = scaler.fit_transform(X[['Amount']])
X['Time'] = scaler.fit_transform(X[['Time']])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

print("Train set:", Counter(y_train))
print("Test set:", Counter(y_test))


Liczba brakujących wartości: 0
Rozkład klas:
 Class
0    284315
1       492
Name: count, dtype: int64
Train set: Counter({0: 199020, 1: 344})
Test set: Counter({0: 85295, 1: 148})


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95, svd_solver='full', random_state=42)
pca.fit(X_train)

print(f"Liczba komponentów PCA potrzebna do wyjaśnienia 95% wariancji: {pca.n_components_}")

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)


Liczba komponentów PCA potrzebna do wyjaśnienia 95% wariancji: 23


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

over = SMOTE(sampling_strategy=0.1, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=42)

resample_pipeline = ImbPipeline([
    ('over', over),
    ('under', under)
])

X_train_bal, y_train_bal = resample_pipeline.fit_resample(X_train_pca, y_train)

print("Po zbilansowaniu:", Counter(y_train_bal))


Po zbilansowaniu: Counter({0: 39804, 1: 19902})


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, confusion_matrix

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

results = {}
for name, model in models.items():
    model.fit(X_train_bal, y_train_bal)
    y_pred = model.predict(X_test_pca)
    y_proba = model.predict_proba(X_test_pca)[:, 1] if hasattr(model, "predict_proba") else y_pred

    rocauc = roc_auc_score(y_test, y_proba)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    cm = confusion_matrix(y_test, y_pred)

    results[name] = {"ROC AUC": rocauc, "Precision": precision, "Recall": recall, "F1": f1}
    print(f"\nModel: {name}")
    print(f"ROC AUC={rocauc:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")
    print("Macierz pomyłek:\n", cm)

best_model = max(results.items(), key=lambda kv: kv[1]['F1'])
print("\nNajlepszy model:", best_model[0])
print("Jego metryki:", best_model[1])



Model: LogisticRegression
ROC AUC=0.9635, Precision=0.0565, Recall=0.8649, F1=0.1060
Macierz pomyłek:
 [[83157  2138]
 [   20   128]]

Model: RandomForest
ROC AUC=0.9634, Precision=0.6833, Recall=0.8311, F1=0.7500
Macierz pomyłek:
 [[85238    57]
 [   25   123]]

Model: KNN
ROC AUC=0.9308, Precision=0.2079, Recall=0.8581, F1=0.3347
Macierz pomyłek:
 [[84811   484]
 [   21   127]]

Najlepszy model: RandomForest
Jego metryki: {'ROC AUC': np.float64(0.9634136217230185), 'Precision': 0.6833333333333333, 'Recall': 0.831081081081081, 'F1': 0.75}


Najlepszym klasyfikatorem jest Random Forest, ponieważ:
* uzyskał najwyższy F1-score (0.75) – najlepszy balans między precision i recall
* ma bardzo wysoką precyzję (68%), więc zgłaszane fraudy w większości są rzeczywistymi oszustwami
* osiągnął również dobry ROC AUC (0.9634), czyli skutecznie rozróżnia obie klasy

Random Forest najlepiej radzi sobie z wykrywaniem oszustw, zapewniając wysoki poziom bezpieczeństwa przy niskiej liczbie fałszywych alarmów.
